In [1]:
import pandas
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import cv2
import os
from nltk import wordpunct_tokenize
import re

In [6]:
!gdown 1hUqu1mbFeTEfBvl-7fc56fHFfCSzIktD

/bin/bash: gdown: command not found


In [5]:
!unzip -qq ml1m.zip -d ml1m

unzip:  cannot find or open ml1m.zip, ml1m.zip.zip or ml1m.zip.ZIP.


In [12]:

users = pandas.read_csv('dataset/users.dat', sep='::',
                        engine='python',
                        names=['userid', 'gender', 'age', 'occupation', 'zip']).set_index('userid')
ratings = pandas.read_csv('dataset/ratings.dat', engine='python',
                          sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])
movies_train = pandas.read_csv('dataset/movies_train.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')
movies_test = pandas.read_csv('dataset/movies_test.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')
movies_train['genre'] = movies_train.genre.str.split('|')
movies_test['genre'] = movies_test.genre.str.split('|')

users.age = users.age.astype('category')
users.gender = users.gender.astype('category')
users.occupation = users.occupation.astype('category')
ratings.movieid = ratings.movieid.astype('category')
ratings.userid = ratings.userid.astype('category')


In [13]:
movies_train
users

,gender,age,occupation,zip
userid,,,,
1,F,1,10,48067
2,M,56,16,70072
3,M,25,15,55117
4,M,45,7,02460
5,M,25,20,55455
...,...,...,...,...
6036,F,25,15,32603
6037,F,45,1,76006
6038,F,56,1,14706


## Dataset + Dataloader

In [16]:
# movies_train = pandas.read_csv('ml1m/dataset/movies_train.dat', engine='python',
#                          sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')

folder_img_path = 'dataset/ml1m-images'
movies_train['id'] = movies_train.index
movies_train.reset_index(inplace=True)
movies_train['img_path'] = movies_train.apply(lambda row: os.path.join(folder_img_path, f'{row.id}.jpg'), axis = 1)
movies_train

,level_0,index,movieid,title,genre,id,img_path
0,0,0,1650,Washington Square (1997),[Drama],0,dataset/ml1m-images/0.jpg
1,1,1,185,"Net, The (1995)","[Sci-Fi, Thriller]",1,dataset/ml1m-images/1.jpg
2,2,2,1377,Batman Returns (1992),"[Action, Adventure, Comedy, Crime]",2,dataset/ml1m-images/2.jpg
3,3,3,3204,"Boys from Brazil, The (1978)",[Thriller],3,dataset/ml1m-images/3.jpg
4,4,4,1901,Dear Jesse (1997),[Documentary],4,dataset/ml1m-images/4.jpg
...,...,...,...,...,...,...,...
3101,3101,3101,2539,Analyze This (1999),[Comedy],3101,dataset/ml1m-images/3101.jpg
3102,3102,3102,3038,"Face in the Crowd, A (1957)",[Drama],3102,dataset/ml1m-images/3102.jpg
3103,3103,3103,1832,Heaven's Burning (1997),"[Action, Drama]",3103,dataset/ml1m-images/3103.jpg
3104,3104,3104,657,Yankee Zulu (1994),"[Comedy, Drama]",3104,dataset/ml1m-images/3104.jpg


In [18]:
folder_img_path = 'dataset/ml1m-images'
movies_test['id'] = movies_test.index
movies_test.reset_index(inplace=True)
movies_test['img_path'] = movies_test.apply(lambda row: os.path.join(folder_img_path, f'{row.id}.jpg'), axis = 1)
movies_test

,index,movieid,title,genre,id,img_path
0,0,3397,"Great Muppet Caper, The (1981)","[Children's, Comedy]",0,dataset/ml1m-images/0.jpg
1,1,2067,Doctor Zhivago (1965),"[Drama, Romance, War]",1,dataset/ml1m-images/1.jpg
2,2,2651,Frankenstein Meets the Wolf Man (1943),[Horror],2,dataset/ml1m-images/2.jpg
3,3,2989,For Your Eyes Only (1981),[Action],3,dataset/ml1m-images/3.jpg
4,4,3415,"Mirror, The (Zerkalo) (1975)",[Drama],4,dataset/ml1m-images/4.jpg
...,...,...,...,...,...,...
772,772,2309,"Inheritors, The (Die Siebtelbauern) (1998)",[Drama],772,dataset/ml1m-images/772.jpg
773,773,2421,"Karate Kid, Part II, The (1986)","[Action, Adventure, Drama]",773,dataset/ml1m-images/773.jpg
774,774,3255,"League of Their Own, A (1992)","[Comedy, Drama]",774,dataset/ml1m-images/774.jpg
775,775,974,Algiers (1938),"[Drama, Romance]",775,dataset/ml1m-images/775.jpg


In [ ]:
def tokenize(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    tokens = wordpunct_tokenize(text)
    tokens = tokens[:-1] # remove last token because it is the year which maybe is not useful
    return tokens

def create_vocab():
    df = movies_train.copy()
    arr_title = df['title'].tolist()
    vocab = set()
    for title in arr_title:
        tokens = tokenize(title)
        vocab.update(tokens)
    vocab = list(vocab)
    pad_token = '<PAD>'
    unk_token = '<UNK>'
    vocab.append(pad_token)
    vocab.append(unk_token)
    return vocab

In [ ]:
class MLDataset(Dataset):
    def __init__(self, is_train=True):
        if is_train:
            self.data =  movies_train
        else:
            self.data = movies_test
        self.data['title_tokens'] = [tokenize(x) for x in self.data.title]

        # create vocab
        vocab = create_vocab()
        pad_token = '<PAD>'
        unk_token = '<UNK>'
        self.token2idx = {token: idx for idx, token in enumerate(vocab)}

        # Create a binary vector for each word in each sentence
        MAX_LENGTH = 7
        vectors = []
        for title_tokens in self.data.title_tokens.tolist():
            if len(title_tokens) < MAX_LENGTH:
                num_pad = MAX_LENGTH - len(title_tokens)
                for idx in range(num_pad):
                    title_tokens.append(pad_token)
            else:
                title_tokens = title_tokens[:MAX_LENGTH]
            title_vectors = []
            for word in title_tokens:
                binary_vector = np.zeros(len(vocab))
                if word in vocab:
                    binary_vector[self.token2idx[word]] = 1
                else:
                    binary_vector[self.token2idx[unk_token]] = 1
                title_vectors.append(binary_vector)

            vectors.append(np.array(title_vectors))
        self.data['vectors'] = vectors

        # label genre
        with open('ml1m/content/dataset/genres.txt', 'r') as f:
            genre_all = f.readlines()
            genre_all = [x.replace('\n','') for x in genre_all]
        self.genre2idx = {genre:idx for idx, genre in enumerate(genre_all)}

    def __getitem__(self, index):
        title = self.data.iloc[index].title
        img_path = self.data.iloc[index].img_path
        genre = self.data.iloc[index].genre

        # preprocess text
        title_vector = self.data.iloc[index].vectors
        title_tensor = torch.from_numpy(title_vector).float()

        # preprocess img
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
        else:
            img = np.random.rand(256,256,3)
        img = cv2.resize(img, (256,256))
        img_tensor = torch.from_numpy(img.transpose(2,0,1)).float()

        # preprocess label
        genre_vector = np.zeros(len(self.genre2idx))

        for g in genre:
            genre_vector[self.genre2idx[g]] = 1
        genre_tensor = torch.from_numpy(genre_vector).float()

        return title_tensor, img_tensor, genre_tensor

    def __len__(self):
        return len(self.data)

In [ ]:
train_set = MLDataset(is_train=True)
test_set = MLDataset(is_train=False)

BATCH_SIZE = 8
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE)

In [ ]:
for title_tensor, img_tensor, genre_tensor in train_dataloader:
    print(title_tensor.shape, img_tensor.shape, genre_tensor.shape)
    break
for title_tensor, img_tensor, genre_tensor in test_dataloader:
    print(title_tensor.shape, img_tensor.shape, genre_tensor.shape)
    break

torch.Size([8, 7, 3899]) torch.Size([8, 3, 256, 256]) torch.Size([8, 18])
torch.Size([8, 7, 3899]) torch.Size([8, 3, 256, 256]) torch.Size([8, 18])


## Model

In [ ]:
with open('ml1m/content/dataset/genres.txt', 'r') as f:
    genre_all = f.readlines()
num_classes = len(genre_all)
num_classes

18

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes, len_vocab, embedding_dimension=3898, hidden_size=64):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size

        # img
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=4, stride=4)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 64 * 64, hidden_size)


        # text
        self.fc2 = nn.Linear(7*len_vocab, hidden_size)

        self.fc3 = nn.Linear(hidden_size*2, num_classes)

    def forward(self, text_tens, img_tens):
        text_feat = self.fc2(self.flatten(text_tens))

        img_feat = self.conv1(img_tensor)
        img_feat = self.maxpool(img_feat)
        img_feat = self.conv2(img_feat)
        img_feat = self.fc1(self.flatten(img_feat))

        out = self.fc3(torch.concat([text_feat, img_feat], dim=1))
        return out

In [ ]:
vocab = create_vocab()
model = BaseModel(num_classes, len(vocab))

out = model(title_tensor, img_tensor)
out.shape

torch.Size([8, 18])

## Loss

In [ ]:
from torch import optim
criterion = nn.CrossEntropyLoss()

learning_rate = 1e-3
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=learning_rate,
)

## Training

In [ ]:
device = torch.device('cuda:0')
NUM_EP = 10
model.to(device)
for ep in range(NUM_EP):
    print('ep:', ep)
    for idx, (title_tensor, img_tensor, genre_tensor) in enumerate(train_dataloader):
        title_tensor = title_tensor.to(device)
        img_tensor = img_tensor.to(device)
        genre_tensor = genre_tensor.to(device)

        out = model(title_tensor, img_tensor)

        loss = criterion(out, genre_tensor)

        loss.backward()
        optimizer.zero_grad()
        optimizer.step()


ep: 0
ep: 1
ep: 2
ep: 3
ep: 4
ep: 5
ep: 6
ep: 7
ep: 8
ep: 9


In [ ]:
!pip install -q torchmetrics
from torchmetrics.classification import MultilabelF1Score

In [ ]:
N, C = genre_tensor.shape

auroc_all = 0
f1_all = 0
f1 = MultilabelF1Score(num_labels=C, threshold=0.5, average='macro')
f1 = f1.to(device)
for title_tensor, img_tensor, genre_tensor in test_dataloader:
    title_tensor = title_tensor.to(device)
    img_tensor = img_tensor.to(device)
    genre_tensor = genre_tensor.to(device)

    out = model(title_tensor, img_tensor)
    f1_batch = f1(out, genre_tensor)
    f1_all += f1_batch

print('F1: ', f1_all/len(test_dataloader))


F1:  tensor(0.0942, device='cuda:0')
